In [29]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

service = Service(executable_path=r'C:\Users\DELL\Documents\web_scraping\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.get("https://www.airbnb.com/")
driver.maximize_window()

element_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[text()="Where"]'))
)
element_to_click.click()

# Locate search input, click, write, and press Enter
search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "bigsearch-query-location-input"))
)
search_input.click()
search_input.send_keys("germany")
search_input.send_keys(Keys.ENTER)

# Locate and click search button
search_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-testid="structured-search-input-search-button"]'))
)
search_button.click()
time.sleep(3)

show_filters_button = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-testid="category-bar-filter-button"]'))
)
show_filters_button.click()

popup = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Filters"]'))
)
driver.execute_script("arguments[0].scrollIntoView();", popup)

# Locate and click "Room"
room_span = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, '//span[text()="Room"]'))
)
room_span.click()

# Locate and adjust slider value
slider = driver.find_element(By.CSS_SELECTOR, 'input[role="slider"]')
ActionChains(driver).click_and_hold(slider).move_by_offset(50, 0).release().perform()

# Locate and click "show" in footer
try:
    show_link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(text(),"Show")]'))
    )
    show_link.click()
except TimeoutException:
    print("Error: 'Show' link not found or not clickable within timeout.")

# card_container = driver.find_elements(By.XPATH, "//div[@data-testid='card-container']")

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new cards to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
card_container = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.XPATH, "//div[@data-testid='card-container']"))
)

print(f"Found {len(card_container)} cards")

address = []
room_descriptions = []
price = []
host_rating = []

for card in card_container:
    # card Title
    card_title_elements = card.find_elements(By.XPATH, './/div[@data-testid="listing-card-title"]')
    if card_title_elements:
        address.append(card_title_elements[0].text.strip())
    else:
        address.append('None')
    
    # Description
    desc_elements = card.find_elements(By.XPATH, './/span[@data-testid="listing-card-name"]')
    if desc_elements:
        room_descriptions.append(desc_elements[0].text.strip())
    else:
        room_descriptions.append('None')
    
    # Stars
    price_elements = card.find_elements(By.XPATH, './/div//span[contains(text(),"per")]')
    if price_elements:
        price.append(price_elements[0].text.strip())
    else:
        price.append('None')
    
    # Forks
    host_elements = card.find_elements(By.XPATH, './/div//span[contains(text(),"reviews")]')
    if host_elements:
        host_rating.append(host_elements[0].text.strip())
    else:
        host_rating.append('None')

print(f"Addresses: {address}")
print(f"Descriptions: {room_descriptions}")
print(f"Prices: {price}")
print(f"Host Ratings: {host_rating}")

df = pd.DataFrame({
    'Address': address,
    'Description': room_descriptions,
    'Price': price,
    'Host Rating': host_rating
})

driver.quit()
df.to_csv('airbnb.csv', index=False)
print(df)

Found 18 cards
Addresses: ['Room in Bad Herrenalb', 'Room in Nürnberg', 'Room in Weissenburg in Bayern', 'Room in Baden-Baden', 'Room in Drestedt', 'Room in Falkensee', 'Room in Schkeuditz', 'Room in Leipzig', 'Room in Buch am Wald', 'Room in Tostedt', 'Room in Grebenau', 'Room in Hamburg', 'Room in Friedrichshafen', 'Room in Dünfus', 'Room in Hamburg', 'Room in Edingen-Neckarhausen', 'Room in Hamburg', 'Room in Göttingen']
Descriptions: ['Room 6 guest rooms in the North Black Forest', 'Small room in shared flat with garden', 'Experienced in the middle of Bavaria', 'In a fairy tale: "Rübezahl" in Zehnthaus Steinbach', 'Room in luxurious house in Drestedt (Room 2)', 'Private room with Queensize Bed', 'Cozy beamed room incl. your own bathroom', 'Private room in the west of Leipzig', 'Kl. room, 1st floor, private bathroom, separate entrance', 'Quiet room in Tostedt', 'cosy room in Gründchen', 'Close to the airport !', 'colourful room looking on an apple orchard', 'Shared apartment', 'Sonn